# MAX + TF.js: Image Segmenter

https://github.com/IBM/MAX-Image-Segmenter

notes/links/etc

- `Netron`: https://github.com/lutzroeder/Netron
- `TF.js Converter`: https://github.com/tensorflow/tfjs-converter
- `TF.js API`: https://js.tensorflow.org/api/latest


<br>

### Table of Contents

1. [Prerequisites](#Prerequisites)
1. [Print the graph nodes](#Print-the-graph-nodes)
1. [Run inference using the graph](#Run-inference-using-the-graph)
1. [Convert the model to a web-friendly format](#Convert-the-model-to-a-web-friendly-format)


<br>

## Prerequisites

- Download and extract the model artifacts for the Image Segmenter:

    [http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/deeplab/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz](http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/deeplab/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz)

    For example, from a terminal window:
    
    ```
    curl -O http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/deeplab/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz

    tar -zxvf deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz
    ```

<br>


In [ ]:
# This notebook has been tested with Python version 3.6.6
!python --version


In [ ]:
# This notebook has been tested with tensorflow 1.12.0, tensorflowjs 0.8.0, and numpy 1.15.1
!pip show tensorflow tensorflowjs numpy


In [ ]:
# Uncomment to install the packages needed

# !pip install -Iv tensorflow
# !pip install -Iv tensorflowjs
# !pip install -Iv numpy

# Restart the kernel after installation completes.


<br>

<strong>NOTE</strong>: Update the variables with the appropriate directory path


In [ ]:
# # full path to extracted frozen graph
frozen_graph_path = '/Users/va/Desktop/max/image-segmenter/model/frozen_inference_graph.pb'


<br>
<hr>

# Print the graph nodes


In [ ]:
import tensorflow as tf

print('TF versions:', tf.GIT_VERSION, tf.VERSION)


<br>

Load the graph

In [ ]:
# load the frozen file and parse it to get the unserialized graph_def
def load_frozen_graph(graph_path):
    with tf.gfile.GFile(graph_path, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        return graph_def


In [ ]:
# load graph
restored_graph_def = load_frozen_graph(frozen_graph_path)


<br>

Print graph nodes

In [ ]:
# print list graph nodes/tensors
def list_nodes(graph_def):
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name=""
        )

    sess = tf.Session(graph=graph)
    nodes = sess.graph.as_graph_def().node
    print('graph has {} nodes \r\n'.format(len(nodes)))
    
    for n in nodes:
        print(n.name + '=>' +  n.op)


In [ ]:
# list graph nodes
list_nodes(restored_graph_def)


<br>
<hr>

# Run inference using the graph


<br>

<strong>NOTE</strong>: Update the variables with the appropriate path to an image to test


In [ ]:
# full path to a test image
test_image_path = '/Users/va/Desktop/max/test/img-03.jpg'


<br>

Resize and display the image


In [ ]:
from PIL import Image

# value to resize image
IMAGE_SIZE = 512

# resizes an image
def resize_image(image_path):
    image = Image.open(image_path)

    width, height = image.size
    resize_ratio = 1.0 * IMAGE_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized = image.convert('RGB').resize(target_size, Image.ANTIALIAS)

    return resized


In [ ]:
# resize the image
resized_image = resize_image(test_image_path)

# display image
resized_image


<br>

Run prediction on the image

In [ ]:
import numpy as np

INPUT_TENSOR_NAME = 'ImageTensor:0'
OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'

# runs prediction
def run_inference(graph_def, image):
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name=""
        )

    sess = tf.Session(graph=graph)

    batch_seg_map = sess.run(
        OUTPUT_TENSOR_NAME,
        feed_dict = { INPUT_TENSOR_NAME: [np.asarray(resized_image)] }
    )
    
    return batch_seg_map[0]


In [ ]:
# run prediction
seg_map = run_inference(restored_graph_def, resized_image)


<br>

Visualize the segments


In [ ]:
# adds color defined by the dataset colormap to the label
def label_to_color_image(label):
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_pascal_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]


# creates a label colormap used in PASCAL VOC segmentation benchmark
def create_pascal_label_colormap():
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)

    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
        ind >>= 3

    return colormap


In [ ]:
# map results to color
seg_image = label_to_color_image(seg_map).astype(np.uint8)

# display results
Image.fromarray(seg_image)


<br>
<hr>

# Convert the model to a web-friendly format

[https://github.com/tensorflow/tfjs-converter](https://github.com/tensorflow/tfjs-converter)


```
tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names='SemanticPredictions' \
    /path/to/frozen/model.pb \
    /path/to/web_asset_output_dir
```


In [ ]:
# check for tensorflowjs_converter
!tensorflowjs_converter --version


<br>

<strong>NOTE</strong>: Update the variables with the appropriate graph output nodes and path to save the converted model assets

In [ ]:
# set the graph output nodes
graph_output_nodes = 'SemanticPredictions'

# set appropriate desired output path for web format
web_asset_dir = '/Users/va/Desktop/max/image-segmenter/model-tfjs'


In [ ]:
import pathlib

# create directory if it does not exist
pathlib.Path(web_asset_dir).mkdir(parents=True, exist_ok=True)


<br>

Run the converter


In [ ]:

!tensorflowjs_converter \
    --input_format=tf_frozen_model \
    --output_node_names={graph_output_nodes} \
    {frozen_graph_path} \
    {web_asset_dir}


<br>

Print the converted model assets


In [ ]:
import os
import time

print("Web asset directory {}:".format(web_asset_dir))

web_assets = os.listdir(web_asset_dir)
web_assets.sort()

for file in web_assets:
    file_stat = os.stat("{}/{}".format(web_asset_dir,file))
    print(" {} {} {:>20}".format(file.ljust(30), time.ctime(file_stat.st_mtime), file_stat.st_size))
